# Tamas Flesch Thesis - EDA step

## LMJU - UpGrad - DS

### Fifa 23 Ultimate Team player price prediction based on the player's attributes

Table of contents
1. Read data
2. Clean data
    - Price_Variation
        - convert the field to numeric
    - Height (extract height in cm into a new column)
        - a few rows were droped because of missing data
    - BodyType (extrct 2 new column, body type text and weight in kg)
        - a few rows were droped because of missing data
    - Club (create new column for ICON players 1/0)
    - League
        - kept the 5 major european league, MLS and 2 special league (ICONS, World cup)
    - Nation
        - kept the 4 country with the most player count which almost have 1000+ players
    - Card_Version
        - removed the column, has mixed values
    - Position columns
        - created 3 separate columns for the alternate positions (those could be NaN if there are no alt positions)
3. Categorical dummy columns
    - create dummy category columns for the normal categorical columns
    - handle alt position columns, these are have missing values as well
4. Export the final dataset
5. visualization TODO!!!
    - outliers check
    - correlation matrix
    - https://towardsdatascience.com/an-extensive-guide-to-exploratory-data-analysis-ddd99a03199e

### 1. Read data

#### Imports

In [1]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Pandas and NumPy
import pandas as pd, numpy as np

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
# Importing all datasets
futbin_data = pd.read_csv("fifa23_players_2023-05-30.csv")
futbin_data.head()

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Price_Variation,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats
0,Pelé,FUT ICONS,Brazil,Icons,98,CAM,"CF,ST",Icon,Explosive,3270000.0,0.96%,5,4,H,M,95.0,96,93,96,60,76,"173cm | 5'8""",Unique (70kg),3928,516,2513
1,Lionel Messi,Paris SG,Argentina,Ligue 1,98,RW,RM,TOTY,Controlled,4350000.0,1.40%,4,4,H,L,93.0,98,97,99,40,77,"169cm | 5'7""",Messi (67kg),5575,504,2469
2,Lionel Messi,Paris SG,Argentina,Ligue 1,98,ST,"RM,RW",TOTS,Controlled,4640000.0,0.30%,4,4,H,L,94.0,97,96,99,40,79,"169cm | 5'7""",Messi (67kg),701,505,2455
3,Karim Benzema,Real Madrid,France,LaLiga Santander,97,CF,ST,TOTY,Controlled,1850000.0,0.54%,4,5,H,M,92.0,97,90,94,45,90,"185cm | 6'1""",Average (81kg),4536,508,2449
4,Kylian Mbappé,Paris SG,France,Ligue 1,97,ST,"CF,LW",TOTY,Controlled,9750000.0,5.15%,5,4,H,L,99.0,96,88,98,44,87,"182cm | 6'0""",Unique (73kg),3548,512,2455


#### check and understand the Price data column

In [5]:
futbin_data.Price.describe()

count    7.160000e+03
mean     2.370740e+04
std      3.146186e+05
min      0.000000e+00
25%      2.000000e+02
50%      3.000000e+02
75%      9.000000e+02
max      1.500000e+07
Name: Price, dtype: float64

In [6]:
len(futbin_data[futbin_data.Price == 0])

751

In [7]:
futbin_data[futbin_data.Price == 0].head()

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Price_Variation,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats
84,Kaoru Mitoma,Brighton,Japan,Premier League,92,LW,"LWB,LM",TOTS,Explosive,0.0,0,4,4,H,M,97.0,88,88,94,76,83,"178cm | 5'10""",Average (71kg),2098,526,2476
136,Gayà,Valencia CF,Spain,LaLiga Santander,91,LB,"LWB,LM",TOTS,Explosive,0.0,0,4,4,H,M,93.0,70,86,91,87,85,"172cm | 5'8""",Lean (66kg),222,512,2398
137,Yannick Carrasco,Atlético de Madrid,Belgium,LaLiga Santander,91,LM,LW,Flashback SBC,Controlled,0.0,0,4,4,H,H,95.0,88,85,94,58,76,"185cm | 6'1""",High & Average+ (73kg),580,496,2373
178,Amine Gouiri,Rennes,France,Ligue 1,90,LM,"CF,ST,LW",Fantasy Objective,Controlled,0.0,0,4,3,H,M,94.0,90,88,92,75,83,"180cm | 5'11""",Average (72kg),1511,522,2487
185,Frank Lampard,FUT ICONS,England,Icons,90,CM,"CDM,CAM",Trophy Titans - ICON,Lengthy,0.0,0,3,4,H,M,74.0,90,88,84,73,81,"184cm | 6'0""",Average (89kg),-147,490,2361


SBC concept, ingame, there is a mini game, where players can finish squad building challenges (SBCs), and get valuable player. These players are not in the market, so players can't buy them on the market, that's why they havn't got a price. So I'll remove these cards from the dataset.

In [8]:
futbin_data = futbin_data[futbin_data.Price > 0]

In [9]:
len(futbin_data[futbin_data.Price == 0])

0

### 2. Clean data

We need only numeric data, so first of all I'll need to investigate the columns, extract the numeric fields, like the height or the body weight. 

Then create dummy columns for the categorical columns like the run style. 

At the last step, I'll create some derived columns where I need to merge some information from columns with a lot of different text values. Like the club or the card version. These columns have a lot of different, distinct values, so I will select some of the most valueable categories and will create a special coulmn with 1/0 values. 1 for the most special categories and 0 for the other columns.

#### Price_Variation

In [10]:
futbin_data.Price_Variation.value_counts()

0            3613
2             844
1             596
12.50%        154
3             139
5.88%          92
10.00%         42
11.11%         39
18.18%         17
16.67%         16
4              16
6.25%          15
4.76%          13
9.09%          13
5.26%          13
5.56%          12
2.33%          11
7.14%          11
8.33%          10
6.67%           9
5.00%           9
2.44%           9
14.29%          8
2.22%           8
9.52%           8
2.08%           7
28.57%          7
35.71%          7
3.33%           7
2.13%           7
2.53%           7
3.13%           7
7.69%           7
3.70%           7
25.00%          7
3.51%           7
0.75%           7
6.10%           7
20.00%          7
22.22%          6
4.17%           6
4.08%           6
0.71%           6
11.76%          6
2.67%           5
5.08%           5
2.63%           5
0.74%           5
1.35%           5
4.00%           5
1.67%           5
8.24%           5
1.49%           5
26.67%          5
3.16%           5
2.50%     

too much missing valuees, and based on the differences, it seems to me that this column has a lot of outliners as well, I think it is better to remove it from the dataset.

In [11]:
futbin_data = futbin_data.drop(['Price_Variation'], 1)

#### Height

In [12]:
futbin_data.head()["Height"][0]

'173cm | 5\'8"'

In [13]:
futbin_data.Height.info()

<class 'pandas.core.series.Series'>
Int64Index: 6409 entries, 0 to 7159
Series name: Height
Non-Null Count  Dtype 
--------------  ----- 
6409 non-null   object
dtypes: object(1)
memory usage: 100.1+ KB


In [14]:
futbin_data.Height.value_counts()

180cm | 5'11"    481
185cm | 6'1"     473
178cm | 5'10"    419
183cm | 6'0"     375
175cm | 5'9"     370
188cm | 6'2"     330
182cm | 6'0"     293
186cm | 6'1"     277
184cm | 6'0"     274
181cm | 5'11"    244
187cm | 6'2"     243
176cm | 5'9"     230
177cm | 5'10"    227
179cm | 5'10"    221
190cm | 6'3"     220
173cm | 5'8"     204
191cm | 6'3"     176
174cm | 5'9"     154
189cm | 6'2"     152
170cm | 5'7"     145
172cm | 5'8"     135
192cm | 6'4"     128
193cm | 6'4"     107
171cm | 5'7"      84
194cm | 6'4"      77
168cm | 5'6"      66
195cm | 6'5"      51
169cm | 5'7"      47
196cm | 6'5"      43
167cm | 5'6"      30
166cm | 5'5"      23
197cm | 6'6"      22
198cm | 6'6"      22
165cm | 5'5"      19
163cm | 5'4"       9
201cm | 6'7"       7
164cm | 5'5"       7
200cm | 6'7"       5
199cm | 6'6"       5
162cm | 5'4"       4
206cm | 6'9"       3
204cm | 6'8"       2
161cm | 5'3"       2
202cm | 6'8"       1
158cm | 5'2"       1
160cm | 5'3"       1
Name: Height, dtype: int64

In [15]:
# remove 0 values, 4 records

In [16]:
futbin_data.shape[0]

6409

In [17]:
futbin_data[futbin_data.Height == "0"]

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats


In [18]:
futbin_data = futbin_data.drop(futbin_data[futbin_data.Height == "0"].index)

In [19]:
futbin_data.shape[0]

6409

In [20]:
futbin_data[futbin_data.Height == "0"]

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats


In [21]:
futbin_data['Height_in_cm'] = futbin_data['Height'].str[:3]

In [22]:
futbin_data['Height_in_cm'].info()

<class 'pandas.core.series.Series'>
Int64Index: 6409 entries, 0 to 7159
Series name: Height_in_cm
Non-Null Count  Dtype 
--------------  ----- 
6409 non-null   object
dtypes: object(1)
memory usage: 100.1+ KB


In [23]:
futbin_data['Height_in_cm'] = pd.to_numeric(futbin_data['Height_in_cm'])

In [24]:
futbin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6409 entries, 0 to 7159
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    6409 non-null   object 
 1   Club                    6409 non-null   object 
 2   Nation                  6409 non-null   object 
 3   League                  6409 non-null   object 
 4   Rating                  6409 non-null   int64  
 5   Main_Position           6409 non-null   object 
 6   Alternate_Positions     6409 non-null   object 
 7   Card_Version            6409 non-null   object 
 8   Run_Style               6409 non-null   object 
 9   Price                   6409 non-null   float64
 10  Skills_Star             6409 non-null   int64  
 11  Weak_Foot_Star          6409 non-null   int64  
 12  Attack_Workrate         6409 non-null   object 
 13  Defense_Workrate        6409 non-null   object 
 14  Pace / Diving           6409 non-null   

#### Body type

In [25]:
futbin_data.BodyType.info()

<class 'pandas.core.series.Series'>
Int64Index: 6409 entries, 0 to 7159
Series name: BodyType
Non-Null Count  Dtype 
--------------  ----- 
6409 non-null   object
dtypes: object(1)
memory usage: 100.1+ KB


In [26]:
futbin_data.BodyType.value_counts()

Average  (70kg)                223
Average  (75kg)                219
Average  (73kg)                178
Average  (72kg)                174
High & Average  (80kg)         171
Average  (74kg)                160
Lean  (70kg)                   148
Average  (76kg)                137
Average  (77kg)                135
Lean  (72kg)                   123
High & Average  (78kg)         121
High & Average  (82kg)         119
Average  (71kg)                118
Lean  (73kg)                   113
High & Average  (83kg)         112
High & Average  (85kg)         109
Lean  (68kg)                   106
Average  (78kg)                106
Lean  (75kg)                   102
Average  (68kg)                 99
Average  (79kg)                 97
High & Average  (84kg)          96
Lean  (74kg)                    95
Average  (80kg)                 90
High & Average  (81kg)          87
Lean  (67kg)                    82
Lean  (65kg)                    81
High & Average  (79kg)          75
High & Average+  (80

I need to create two more columns from this one. The first one would be a numeric field in kg. The second is a categorical field with the body type.

In [27]:
futbin_data[['BodyType_Text', 'BodyType_Weight']] = futbin_data['BodyType'].str.split('  ', 1, expand=True)

First try, using '  ' (double spaces), let's try, is it working?

In [28]:
futbin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6409 entries, 0 to 7159
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    6409 non-null   object 
 1   Club                    6409 non-null   object 
 2   Nation                  6409 non-null   object 
 3   League                  6409 non-null   object 
 4   Rating                  6409 non-null   int64  
 5   Main_Position           6409 non-null   object 
 6   Alternate_Positions     6409 non-null   object 
 7   Card_Version            6409 non-null   object 
 8   Run_Style               6409 non-null   object 
 9   Price                   6409 non-null   float64
 10  Skills_Star             6409 non-null   int64  
 11  Weak_Foot_Star          6409 non-null   int64  
 12  Attack_Workrate         6409 non-null   object 
 13  Defense_Workrate        6409 non-null   object 
 14  Pace / Diving           6409 non-null   

Some rows are missing from the Weight coulmn, maybe double-space is not the right separator for every row.

In [29]:
futbin_data.BodyType_Text.value_counts()

Average               2128
High & Average        1477
Lean                  1313
High & Average+        681
Stocky                 191
Short & Lean-          178
Unique                 128
Short & Lean           128
High & Stocky          118
Short and Balanced      35
Normal                  12
Messi                    6
High & Lean              5
Salah                    2
Ronaldinho               2
R9                       2
CR7                      1
Courtois                 1
Shaqiri                  1
Name: BodyType_Text, dtype: int64

In [30]:
futbin_data.BodyType_Weight.value_counts()

(75kg)     483
(70kg)     443
(80kg)     378
(73kg)     367
(72kg)     351
(74kg)     344
(78kg)     340
(76kg)     327
(77kg)     298
(79kg)     252
(68kg)     243
(71kg)     227
(82kg)     212
(83kg)     181
(85kg)     173
(67kg)     171
(65kg)     171
(69kg)     167
(81kg)     160
(84kg)     149
(66kg)     114
(64kg)      94
(86kg)      86
(88kg)      79
(87kg)      71
(63kg)      68
(90kg)      63
(62kg)      55
(60kg)      49
(89kg)      48
(91kg)      27
(61kg)      27
(92kg)      25
(93kg)      22
(94kg)      17
(58kg)      15
(59kg)      10
 (68kg)     10
(98kg)       8
(57kg)       8
(97kg)       7
 (65kg)      6
(95kg)       6
(96kg)       5
 (70kg)      5
(56kg)       4
(100kg)      3
 (71kg)      3
(99kg)       3
 (75kg)      3
 (72kg)      3
 (73kg)      2
(55kg)       2
 (66kg)      1
(102kg)      1
(49kg)       1
 (67kg)      1
 (63kg)      1
 (80kg)      1
 (62kg)      1
(0kg)        1
 (77kg)      1
 (69kg)      1
 (93kg)      1
(54kg)       1
Name: BodyType_Weight, dt

Second try, sometimes we have 2 or 3 spaces as a separator so I will try a parenthesis (

In [31]:
futbin_data[['BodyType_Text', 'BodyType_Weight']] = futbin_data['BodyType'].str.split('(', 1, expand=True)

In [32]:
futbin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6409 entries, 0 to 7159
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    6409 non-null   object 
 1   Club                    6409 non-null   object 
 2   Nation                  6409 non-null   object 
 3   League                  6409 non-null   object 
 4   Rating                  6409 non-null   int64  
 5   Main_Position           6409 non-null   object 
 6   Alternate_Positions     6409 non-null   object 
 7   Card_Version            6409 non-null   object 
 8   Run_Style               6409 non-null   object 
 9   Price                   6409 non-null   float64
 10  Skills_Star             6409 non-null   int64  
 11  Weak_Foot_Star          6409 non-null   int64  
 12  Attack_Workrate         6409 non-null   object 
 13  Defense_Workrate        6409 non-null   object 
 14  Pace / Diving           6409 non-null   

In [33]:
futbin_data.BodyType_Text.value_counts()

Average                  2128
High & Average           1477
Lean                     1313
High & Average+           681
Stocky                    191
Short & Lean-             178
Unique                    128
Short & Lean              128
High & Stocky             118
Short and Balanced         35
Normal                     12
Messi                       6
High & Lean                 5
Salah                       2
Ronaldinho                  2
R9                          2
CR7                         1
Courtois                    1
Shaqiri                     1
Name: BodyType_Text, dtype: int64

In [34]:
futbin_data.BodyType_Weight.value_counts()

75kg)     486
70kg)     448
80kg)     379
73kg)     369
72kg)     354
74kg)     344
78kg)     340
76kg)     327
77kg)     299
68kg)     253
79kg)     252
71kg)     230
82kg)     212
83kg)     181
65kg)     177
85kg)     173
67kg)     172
69kg)     168
81kg)     160
84kg)     149
66kg)     115
64kg)      94
86kg)      86
88kg)      79
87kg)      71
63kg)      69
90kg)      63
62kg)      56
60kg)      49
89kg)      48
61kg)      27
91kg)      27
92kg)      25
93kg)      23
94kg)      17
58kg)      15
59kg)      10
57kg)       8
98kg)       8
97kg)       7
95kg)       6
96kg)       5
56kg)       4
100kg)      3
99kg)       3
55kg)       2
0kg)        1
49kg)       1
102kg)      1
54kg)       1
Name: BodyType_Weight, dtype: int64

In [35]:
futbin_data.BodyType_Weight.count()

6397

In [36]:
futbin_data[futbin_data.BodyType_Weight.isnull() == True]

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight
21,Müller,FUT ICONS,Germany,Icons,94,ST,CF,Icon,Controlled,139000.0,4,4,H,M,88.0,94,77,88,43,76,"176cm | 5'9""",Normal,1034,466,2287,176,Normal,None
45,Jairzinho,FUT ICONS,Brazil,Icons,93,RW,0,RM,Controlled,4120000.0,5,5,H,H,94.0,92,88,93,52,77,"173cm | 5'8""",Normal,-471,496,2381,173,Normal,None
65,Jairzinho,FUT ICONS,Brazil,Icons,92,RW,"RM,ST",Icon,Controlled,1740000.0,5,4,H,H,94.0,92,87,93,51,76,"173cm | 5'8""",Normal,1176,493,2366,173,Normal,None
243,Ledley King,HERO,England,Premier League,89,CB,0,Fantasy Hero,Lengthy,119000.0,2,4,M,H,83.0,58,72,78,92,86,"188cm | 6'2""",Normal,892,469,2186,188,Normal,None
516,Ledley King,HERO,England,Premier League,86,CB,0,Hero,Lengthy,16000.0,2,4,M,H,80.0,55,69,72,89,80,"188cm | 6'2""",Normal,1718,445,2070,188,Normal,None
836,Lucas Ocampos,Ajax,Argentina,Eredivisie,81,LW,"LM,RW",Normal,Controlled,5300.0,4,3,H,H,80.0,81,77,82,62,83,"188cm | 6'2""",Normal,22,465,2216,188,Normal,None
992,Martin Dúbravka,Newcastle Utd,Slovakia,Premier League,80,GK,0,non-rare,Controlled,550.0,1,2,M,M,78.0,76,79,81,46,77,"190cm | 6'3""",Normal,16,437,1002,190,Normal,None
1586,Jan Bednarek,Southampton,Poland,Premier League,76,CB,0,non-rare,Controlled,450.0,2,2,M,H,48.0,22,53,55,77,78,"189cm | 6'2""",Normal,17,333,1562,189,Normal,None
4769,Sylvester Jasper,Fulham,Bulgaria,Premier League,65,RM,"LM,RW",non-rare,Explosive,350.0,4,4,H,M,81.0,58,59,67,39,50,"172cm | 5'8""",Normal,2,354,1656,172,Normal,None
5506,Harvey Vale,Chelsea,England,Premier League,63,LW,"LM,CAM",non-rare,Explosive,500.0,3,4,H,M,73.0,58,61,65,41,48,"180cm | 5'11""",Normal,5,346,1643,180,Normal,None


In [37]:
futbin_data = futbin_data.drop(futbin_data[futbin_data.BodyType_Weight.isnull() == True].index)

In [38]:
futbin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6397 entries, 0 to 7159
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    6397 non-null   object 
 1   Club                    6397 non-null   object 
 2   Nation                  6397 non-null   object 
 3   League                  6397 non-null   object 
 4   Rating                  6397 non-null   int64  
 5   Main_Position           6397 non-null   object 
 6   Alternate_Positions     6397 non-null   object 
 7   Card_Version            6397 non-null   object 
 8   Run_Style               6397 non-null   object 
 9   Price                   6397 non-null   float64
 10  Skills_Star             6397 non-null   int64  
 11  Weak_Foot_Star          6397 non-null   int64  
 12  Attack_Workrate         6397 non-null   object 
 13  Defense_Workrate        6397 non-null   object 
 14  Pace / Diving           6397 non-null   

In [39]:
futbin_data.BodyType_Text.value_counts()

Average                  2128
High & Average           1477
Lean                     1313
High & Average+           681
Stocky                    191
Short & Lean-             178
Short & Lean              128
Unique                    128
High & Stocky             118
Short and Balanced         35
Messi                       6
High & Lean                 5
Ronaldinho                  2
Salah                       2
R9                          2
CR7                         1
Courtois                    1
Shaqiri                     1
Name: BodyType_Text, dtype: int64

In [40]:
futbin_data.BodyType_Text = futbin_data.BodyType_Text.str.strip()

In [41]:
futbin_data.BodyType_Weight.value_counts()

75kg)     486
70kg)     448
80kg)     379
73kg)     369
72kg)     354
74kg)     344
78kg)     340
76kg)     327
77kg)     299
68kg)     253
79kg)     252
71kg)     230
82kg)     212
83kg)     181
65kg)     177
85kg)     173
67kg)     172
69kg)     168
81kg)     160
84kg)     149
66kg)     115
64kg)      94
86kg)      86
88kg)      79
87kg)      71
63kg)      69
90kg)      63
62kg)      56
60kg)      49
89kg)      48
61kg)      27
91kg)      27
92kg)      25
93kg)      23
94kg)      17
58kg)      15
59kg)      10
57kg)       8
98kg)       8
97kg)       7
95kg)       6
96kg)       5
56kg)       4
100kg)      3
99kg)       3
55kg)       2
0kg)        1
49kg)       1
102kg)      1
54kg)       1
Name: BodyType_Weight, dtype: int64

In [42]:
futbin_data.BodyType_Weight = futbin_data.BodyType_Weight.str[:-3]

In [43]:
futbin_data.BodyType_Weight = pd.to_numeric(futbin_data.BodyType_Weight)

In [44]:
futbin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6397 entries, 0 to 7159
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    6397 non-null   object 
 1   Club                    6397 non-null   object 
 2   Nation                  6397 non-null   object 
 3   League                  6397 non-null   object 
 4   Rating                  6397 non-null   int64  
 5   Main_Position           6397 non-null   object 
 6   Alternate_Positions     6397 non-null   object 
 7   Card_Version            6397 non-null   object 
 8   Run_Style               6397 non-null   object 
 9   Price                   6397 non-null   float64
 10  Skills_Star             6397 non-null   int64  
 11  Weak_Foot_Star          6397 non-null   int64  
 12  Attack_Workrate         6397 non-null   object 
 13  Defense_Workrate        6397 non-null   object 
 14  Pace / Diving           6397 non-null   

In [45]:
futbin_data.head()

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight
0,Pelé,FUT ICONS,Brazil,Icons,98,CAM,"CF,ST",Icon,Explosive,3270000.0,5,4,H,M,95.0,96,93,96,60,76,"173cm | 5'8""",Unique (70kg),3928,516,2513,173,Unique,70
1,Lionel Messi,Paris SG,Argentina,Ligue 1,98,RW,RM,TOTY,Controlled,4350000.0,4,4,H,L,93.0,98,97,99,40,77,"169cm | 5'7""",Messi (67kg),5575,504,2469,169,Messi,67
2,Lionel Messi,Paris SG,Argentina,Ligue 1,98,ST,"RM,RW",TOTS,Controlled,4640000.0,4,4,H,L,94.0,97,96,99,40,79,"169cm | 5'7""",Messi (67kg),701,505,2455,169,Messi,67
3,Karim Benzema,Real Madrid,France,LaLiga Santander,97,CF,ST,TOTY,Controlled,1850000.0,4,5,H,M,92.0,97,90,94,45,90,"185cm | 6'1""",Average (81kg),4536,508,2449,185,Average,81
4,Kylian Mbappé,Paris SG,France,Ligue 1,97,ST,"CF,LW",TOTY,Controlled,9750000.0,5,4,H,L,99.0,96,88,98,44,87,"182cm | 6'0""",Unique (73kg),3548,512,2455,182,Unique,73


Remove extracted columns

In [46]:
futbin_data = futbin_data.drop(['Height','BodyType'], 1)

In [47]:
futbin_data.Club.value_counts()

FUT ICONS                           128
HERO                                 38
Arsenal                              29
Liverpool                            25
Paris SG                             25
Manchester City                      24
Manchester Utd                       24
Real Madrid                          23
Dortmund                             22
RC Celta                             22
Banfield                             21
FC Bayern                            21
Atlético de Madrid                   21
Juventus                             20
Austin FC                            20
Vélez Sarsfield                      20
FC Barcelona                         20
Ajax                                 19
Roma FC                              19
Unión                                19
Newcastle Utd                        18
RB Leipzig                           18
Napoli                               18
Zagłębie Lubin                       18
Suwon Samsung                        18


#### Club

It has many values, I'll create a new column which will have 1 if the club is FUT ICONS or HERO and 0 otherwise.

In [48]:
# Defining the map function
def icon_map(x):
    if x == 'FUT ICONS':
        return 1
    elif x == 'HERO':
        return 1
    else:
        return 0

# Applying the function to the club column
futbin_data['Club_Hero'] = futbin_data['Club'].apply(icon_map)

In [49]:
futbin_data.Club_Hero.value_counts()

0    6231
1     166
Name: Club_Hero, dtype: int64

In [50]:
futbin_data = futbin_data.drop(['Club'], 1)

In [51]:
futbin_data.head()

,Name,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero
0,Pelé,Brazil,Icons,98,CAM,"CF,ST",Icon,Explosive,3270000.0,5,4,H,M,95.0,96,93,96,60,76,3928,516,2513,173,Unique,70,1
1,Lionel Messi,Argentina,Ligue 1,98,RW,RM,TOTY,Controlled,4350000.0,4,4,H,L,93.0,98,97,99,40,77,5575,504,2469,169,Messi,67,0
2,Lionel Messi,Argentina,Ligue 1,98,ST,"RM,RW",TOTS,Controlled,4640000.0,4,4,H,L,94.0,97,96,99,40,79,701,505,2455,169,Messi,67,0
3,Karim Benzema,France,LaLiga Santander,97,CF,ST,TOTY,Controlled,1850000.0,4,5,H,M,92.0,97,90,94,45,90,4536,508,2449,185,Average,81,0
4,Kylian Mbappé,France,Ligue 1,97,ST,"CF,LW",TOTY,Controlled,9750000.0,5,4,H,L,99.0,96,88,98,44,87,3548,512,2455,182,Unique,73,0


#### League

In [52]:
futbin_data.League.value_counts()

Premier League                   306
LaLiga Santander                 295
Major League Soccer              290
EFL Championship (ENG 2)         277
Serie A TIM                      277
Ligue 1                          259
Bundesliga                       254
EFL League One (ENG 3)           234
EFL League Two (ENG 4)           210
CONMEBOL Libertadores            195
LaLiga SmartBank (ESP 2)         190
Chinese FA Super L. (CHN 1)      176
3. Liga (GER 3)                  174
CONMEBOL Sudamericana            174
Bundesliga 2 (GER 2)             171
Eredivisie                       169
Italy Serie B (2)                168
MBS Pro League (SAU 1)           164
Liga NOS (POR 1)                 162
Polski Ekstraklasa (POL 1)       162
1A Pro League (BEL 1)            161
Liga I (ROM 1)                   159
Ligue 2 (FRA 2)                  154
Süper Lig (TUR 1)                153
Eliteserien (NOR 1)              134
Allsvenskan (SWE 1)              134
K League 1 (KOR 1)               130
I

We have many leagues, I'll collect the 5 mayor european leagues, and some special cases (Worls Cup, Icons), and all of the other values will be other. After this simplification I'll create categorical columns from the values.

In [53]:
# Defining the map function
def special_club_map(x):
    if x == 'World Cup':
        return 'World Cup'
    elif x == 'Icons':
        return 'Icons'
    elif x == 'LaLiga Santander':
        return 'LaLiga Santander'
    elif x == 'Premier League':
        return 'Premier League'
    elif x == 'Major League Soccer':
        return 'Major League Soccer'
    elif x == 'Serie A TIM':
        return 'Serie A TIM'
    elif x == 'Ligue 1':
        return 'Ligue 1'
    elif x == 'Bundesliga':
        return 'Bundesliga'
    else:
        return 'Other'

# Applying the function to the club column
futbin_data['League_Cat'] = futbin_data['League'].apply(special_club_map)

In [54]:
futbin_data.League_Cat.value_counts()

Other                  4588
Premier League          306
LaLiga Santander        295
Major League Soccer     290
Serie A TIM             277
Ligue 1                 259
Bundesliga              254
Icons                   128
Name: League_Cat, dtype: int64

In [55]:
futbin_data = futbin_data.drop(['League'], 1)

In [56]:
futbin_data.head()

,Name,Nation,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat
0,Pelé,Brazil,98,CAM,"CF,ST",Icon,Explosive,3270000.0,5,4,H,M,95.0,96,93,96,60,76,3928,516,2513,173,Unique,70,1,Icons
1,Lionel Messi,Argentina,98,RW,RM,TOTY,Controlled,4350000.0,4,4,H,L,93.0,98,97,99,40,77,5575,504,2469,169,Messi,67,0,Ligue 1
2,Lionel Messi,Argentina,98,ST,"RM,RW",TOTS,Controlled,4640000.0,4,4,H,L,94.0,97,96,99,40,79,701,505,2455,169,Messi,67,0,Ligue 1
3,Karim Benzema,France,97,CF,ST,TOTY,Controlled,1850000.0,4,5,H,M,92.0,97,90,94,45,90,4536,508,2449,185,Average,81,0,LaLiga Santander
4,Kylian Mbappé,France,97,ST,"CF,LW",TOTY,Controlled,9750000.0,5,4,H,L,99.0,96,88,98,44,87,3548,512,2455,182,Unique,73,0,Ligue 1


#### Nation

I'll check the nation column, but I have a feeling that I need to remove it, because it has probably a lot of values and it will be hard to extract information from it.

In [57]:
futbin_data.Nation.value_counts()

England                   645
Germany                   424
Spain                     395
France                    363
Argentina                 341
Italy                     253
Netherlands               180
Brazil                    175
China PR                  159
Sweden                    148
Portugal                  143
Romania                   141
Republic of Ireland       140
United States             139
Poland                    138
Norway                    128
Denmark                   126
Korea Republic            123
Belgium                   118
Saudi Arabia              114
Austria                    99
Australia                  95
Scotland                   91
Colombia                   88
Switzerland                81
Turkey                     74
Wales                      63
Uruguay                    62
Croatia                    57
India                      57
Paraguay                   52
Chile                      48
Senegal                    46
Nigeria   

We have a few nations more than 1000 players so I decided that I will keep those, and will use other for the others. It is just a subjective decision, so later it can be modified, and maybe keep other nations as well.

One other derived direction could be to merge the countries by continents Europe, America, Asia, and so on...

In [58]:
# Defining the map function
def nation_keep_map(x):
    if x == 'England':
        return 'England'
    elif x == 'Germany':
        return 'Germany'
    elif x == 'Spain':
        return 'Spain'
    elif x == 'France':
        return 'France'
    elif x == 'Argentina':
        return 'Argentina'
    else:
        return 'Other'

# Applying the function to the club column
futbin_data['Nation_Cat'] = futbin_data['Nation'].apply(nation_keep_map)

In [59]:
futbin_data.Nation_Cat.value_counts()

Other        4229
England       645
Germany       424
Spain         395
France        363
Argentina     341
Name: Nation_Cat, dtype: int64

In [60]:
futbin_data = futbin_data.drop(['Nation'], 1)

In [61]:
futbin_data.head()

,Name,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat
0,Pelé,98,CAM,"CF,ST",Icon,Explosive,3270000.0,5,4,H,M,95.0,96,93,96,60,76,3928,516,2513,173,Unique,70,1,Icons,Other
1,Lionel Messi,98,RW,RM,TOTY,Controlled,4350000.0,4,4,H,L,93.0,98,97,99,40,77,5575,504,2469,169,Messi,67,0,Ligue 1,Argentina
2,Lionel Messi,98,ST,"RM,RW",TOTS,Controlled,4640000.0,4,4,H,L,94.0,97,96,99,40,79,701,505,2455,169,Messi,67,0,Ligue 1,Argentina
3,Karim Benzema,97,CF,ST,TOTY,Controlled,1850000.0,4,5,H,M,92.0,97,90,94,45,90,4536,508,2449,185,Average,81,0,LaLiga Santander,France
4,Kylian Mbappé,97,ST,"CF,LW",TOTY,Controlled,9750000.0,5,4,H,L,99.0,96,88,98,44,87,3548,512,2455,182,Unique,73,0,Ligue 1,France


#### Card_Version

In [62]:
futbin_data.Card_Version.value_counts()

Normal                  4711
non-rare                 353
Rare                     221
Libertadores             175
IF                       165
Sudamericana             139
Icon                      92
TOTS                      43
SIF                       32
CF                        26
Hero                      22
World Cup ICON            18
CM                        16
Fantasy                   16
Winter Wildcards          14
LWB                       12
Fantasy Hero              12
FUT Centurions            11
Flashback SBC             11
RWB                       10
World Cup PTG              9
Out Of Position SBC        8
Trophy Titans - ICON       8
TOTY                       8
CDM                        8
World Cup Phenoms          8
World Cup Showdown S       7
Winter Wildcards SBC       7
FUT Future Stars           7
UCL Live                   7
FUT Birthday               6
OTW                        6
MOTM                       6
Player Moments SBC         5
Out Of Positio

too many options and the values are mixed, not just card versions (toty, fut hero, winter wildcards, etc) but positions (LB, RB, etc and these combinations as well)

I'll just remove this column for now.

In [63]:
futbin_data = futbin_data.drop(['Card_Version'], 1)

In [64]:
futbin_data.head()

,Name,Rating,Main_Position,Alternate_Positions,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat
0,Pelé,98,CAM,"CF,ST",Explosive,3270000.0,5,4,H,M,95.0,96,93,96,60,76,3928,516,2513,173,Unique,70,1,Icons,Other
1,Lionel Messi,98,RW,RM,Controlled,4350000.0,4,4,H,L,93.0,98,97,99,40,77,5575,504,2469,169,Messi,67,0,Ligue 1,Argentina
2,Lionel Messi,98,ST,"RM,RW",Controlled,4640000.0,4,4,H,L,94.0,97,96,99,40,79,701,505,2455,169,Messi,67,0,Ligue 1,Argentina
3,Karim Benzema,97,CF,ST,Controlled,1850000.0,4,5,H,M,92.0,97,90,94,45,90,4536,508,2449,185,Average,81,0,LaLiga Santander,France
4,Kylian Mbappé,97,ST,"CF,LW",Controlled,9750000.0,5,4,H,L,99.0,96,88,98,44,87,3548,512,2455,182,Unique,73,0,Ligue 1,France


#### Positions

In [65]:
futbin_data.Main_Position.value_counts()

CB     1107
ST      921
CM      740
GK      685
CDM     507
CAM     400
RB      396
LB      392
RM      292
LM      285
RW      192
LW      189
RWB     120
LWB      98
CF       73
Name: Main_Position, dtype: int64

would be perfect for categorical columns, will convert it at the last step.

In [66]:
futbin_data.Alternate_Positions.value_counts()

0             2028
CF             653
CM             411
CDM            276
LWB            237
RWB            198
RB             152
LB             106
LM,RW           95
RM              93
CAM             91
RW              89
RM,LW           80
LM              77
LWB,LM          73
LW              70
RWB,RM          61
CF,LW           60
LM,CF           53
RWB,LB          50
CDM,CAM         49
CB              42
ST,LW           41
LM,ST           40
RWB,CB          40
ST              40
CF,RW           35
RM,CF           34
RM,LM           33
RB,RM           32
CAM,LW          31
RW,ST           31
CB,CM           31
CB,LWB          29
LM,RW,ST        29
RM,CAM,LW       29
RM,ST           28
CAM,CF          27
LB,LM           26
RM,ST,LW        26
LM,CAM          25
CAM,RW          23
RW,LW           22
CF,RW,LW        21
CDM,CM          21
RM,CM           21
RM,CAM          21
RM,LM,CF        19
CAM,ST          19
RB,LWB          18
CM,RW           18
CM,LM           15
CM,CF       

In [67]:
futbin_data['Alt_Pos_List'] = futbin_data['Alternate_Positions'].str.split(',')

In [68]:
futbin_data['Alt_Pos_List'].head()

0    [CF, ST]
1        [RM]
2    [RM, RW]
3        [ST]
4    [CF, LW]
Name: Alt_Pos_List, dtype: object

In [69]:
#futbin_data[['Alt_Pos_1', 'Alt_Pos_2', 'Alt_Pos_3']] = futbin_data['Alternate_Positions'].str.split(',', 1, expand=True)

In [70]:
futbin_data = futbin_data.join(futbin_data['Alternate_Positions'].str.split(',', expand=True).rename(columns={0:'Alt_Pos_1', 1:'Alt_Pos_2', 2:'Alt_Pos_3'}))

In [71]:
futbin_data.head()

,Name,Rating,Main_Position,Alternate_Positions,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_List,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3
0,Pelé,98,CAM,"CF,ST",Explosive,3270000.0,5,4,H,M,95.0,96,93,96,60,76,3928,516,2513,173,Unique,70,1,Icons,Other,"[CF, ST]",CF,ST,None
1,Lionel Messi,98,RW,RM,Controlled,4350000.0,4,4,H,L,93.0,98,97,99,40,77,5575,504,2469,169,Messi,67,0,Ligue 1,Argentina,[RM],RM,None,None
2,Lionel Messi,98,ST,"RM,RW",Controlled,4640000.0,4,4,H,L,94.0,97,96,99,40,79,701,505,2455,169,Messi,67,0,Ligue 1,Argentina,"[RM, RW]",RM,RW,None
3,Karim Benzema,97,CF,ST,Controlled,1850000.0,4,5,H,M,92.0,97,90,94,45,90,4536,508,2449,185,Average,81,0,LaLiga Santander,France,[ST],ST,None,None
4,Kylian Mbappé,97,ST,"CF,LW",Controlled,9750000.0,5,4,H,L,99.0,96,88,98,44,87,3548,512,2455,182,Unique,73,0,Ligue 1,France,"[CF, LW]",CF,LW,None


In [72]:
futbin_data['Alt_Pos_Count'] = futbin_data['Alt_Pos_List'].str.len()

In [73]:
futbin_data = futbin_data.drop(['Alternate_Positions'], 1)

In [74]:
futbin_data = futbin_data.drop(['Alt_Pos_List'], 1)

In [75]:
futbin_data.head()

,Name,Rating,Main_Position,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count
0,Pelé,98,CAM,Explosive,3270000.0,5,4,H,M,95.0,96,93,96,60,76,3928,516,2513,173,Unique,70,1,Icons,Other,CF,ST,None,2
1,Lionel Messi,98,RW,Controlled,4350000.0,4,4,H,L,93.0,98,97,99,40,77,5575,504,2469,169,Messi,67,0,Ligue 1,Argentina,RM,None,None,1
2,Lionel Messi,98,ST,Controlled,4640000.0,4,4,H,L,94.0,97,96,99,40,79,701,505,2455,169,Messi,67,0,Ligue 1,Argentina,RM,RW,None,2
3,Karim Benzema,97,CF,Controlled,1850000.0,4,5,H,M,92.0,97,90,94,45,90,4536,508,2449,185,Average,81,0,LaLiga Santander,France,ST,None,None,1
4,Kylian Mbappé,97,ST,Controlled,9750000.0,5,4,H,L,99.0,96,88,98,44,87,3548,512,2455,182,Unique,73,0,Ligue 1,France,CF,LW,None,2


In [76]:
futbin_data.tail()

,Name,Rating,Main_Position,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count
7154,Hao Ning,48,ST,Controlled,200.0,2,3,M,M,64.0,44,55,48,35,47,1,293,1398,175,Stocky,70,0,Other,Other,CF,None,None,1
7155,Jacen Russell-Rowe,48,ST,Controlled,200.0,2,3,M,M,64.0,48,41,53,15,48,6,269,1239,183,Lean,73,0,Major League Soccer,Other,CF,None,None,1
7156,Andre? Brînzea,48,GK,Controlled,200.0,0,2,M,M,51.0,44,46,49,17,48,3,255,603,187,High & Average,76,0,Other,Other,0,None,None,1
7158,Junjie Wu,46,LB,Controlled,200.0,2,3,M,M,55.0,25,29,34,48,57,10,248,1125,188,High & Average,83,0,Other,Other,LWB,None,None,1
7159,Kailin Barlow,46,CDM,Controlled,250.0,0,2,H,M,65.0,36,43,46,43,53,36,286,1323,181,Average,73,0,Other,Other,0,None,None,1


In [77]:
futbin_data.Alt_Pos_1.replace('0', np.nan, inplace=True)

In [78]:
futbin_data.Alt_Pos_2.fillna(value=np.nan, inplace=True)

In [79]:
futbin_data.Alt_Pos_3.fillna(value=np.nan, inplace=True)

Need to update Alt_Pos_Count to 0 if the Alt_Pos_1 is NaN, because originally, the field conatins 0 instead of null value.

In [80]:
#futbin_data['Alt_Pos_Count'] = futbin_data.apply(lambda x: 0 if x['Alt_Pos_1']=='NaN' else x['Alt_Pos_Count'], axis=1)
futbin_data.Alt_Pos_Count = np.where(futbin_data.Alt_Pos_1.isna(), 0, futbin_data.Alt_Pos_Count)

In [81]:
futbin_data.Alt_Pos_1.info()

<class 'pandas.core.series.Series'>
Int64Index: 6397 entries, 0 to 7159
Series name: Alt_Pos_1
Non-Null Count  Dtype 
--------------  ----- 
4369 non-null   object
dtypes: object(1)
memory usage: 358.0+ KB


In [82]:
futbin_data[futbin_data.Alt_Pos_1.isna()].tail()

,Name,Rating,Main_Position,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count
7150,Peinan Li,48,CB,Controlled,200.0,0,3,M,M,50.0,26,29,31,49,54,0,239,1128,185,High & Average,75,0,Other,Other,NaN,NaN,NaN,0
7151,Wenzhe Zhao,48,CB,Controlled,200.0,0,3,M,M,55.0,24,26,30,50,56,7,241,1114,184,Lean,73,0,Other,Other,NaN,NaN,NaN,0
7152,Xiongtao Deng,48,GK,Controlled,200.0,0,3,M,M,48.0,49,53,48,20,50,0,268,540,190,High & Average,78,0,Other,Other,NaN,NaN,NaN,0
7156,Andre? Brînzea,48,GK,Controlled,200.0,0,2,M,M,51.0,44,46,49,17,48,3,255,603,187,High & Average,76,0,Other,Other,NaN,NaN,NaN,0
7159,Kailin Barlow,46,CDM,Controlled,250.0,0,2,H,M,65.0,36,43,46,43,53,36,286,1323,181,Average,73,0,Other,Other,NaN,NaN,NaN,0


In [83]:
futbin_data.tail()

,Name,Rating,Main_Position,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count
7154,Hao Ning,48,ST,Controlled,200.0,2,3,M,M,64.0,44,55,48,35,47,1,293,1398,175,Stocky,70,0,Other,Other,CF,NaN,NaN,1
7155,Jacen Russell-Rowe,48,ST,Controlled,200.0,2,3,M,M,64.0,48,41,53,15,48,6,269,1239,183,Lean,73,0,Major League Soccer,Other,CF,NaN,NaN,1
7156,Andre? Brînzea,48,GK,Controlled,200.0,0,2,M,M,51.0,44,46,49,17,48,3,255,603,187,High & Average,76,0,Other,Other,NaN,NaN,NaN,0
7158,Junjie Wu,46,LB,Controlled,200.0,2,3,M,M,55.0,25,29,34,48,57,10,248,1125,188,High & Average,83,0,Other,Other,LWB,NaN,NaN,1
7159,Kailin Barlow,46,CDM,Controlled,250.0,0,2,H,M,65.0,36,43,46,43,53,36,286,1323,181,Average,73,0,Other,Other,NaN,NaN,NaN,0


### 3. Categorical columns

Main_Position

Run_Style

Attack_Workrate

Defense_Workrate

BodyType_Text

League_Cat

Nation_Cat

======================================================

Alt_Pos_1

Alt_Pos_2

Alt_Pos_3

First I'll handle the "normal" cat columns, skipping the from alt pos, because those could have emty fields.

In [84]:
# Creating a dummy variable for some of the categorical variables and dropping the first one.
dummy_fields = pd.get_dummies(futbin_data[['Main_Position', 'Run_Style', 'Attack_Workrate', 'Defense_Workrate', 
                                           'BodyType_Text', 'League_Cat', 'Nation_Cat']], drop_first=True)

# Adding the results to the master dataframe
futbin_data = pd.concat([futbin_data, dummy_fields], axis=1)

In [85]:
futbin_data.head()

,Name,Rating,Main_Position,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Explosive,Run_Style_Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_High & Average,BodyType_Text_High & Average+,BodyType_Text_High & Lean,BodyType_Text_High & Stocky,BodyType_Text_Lean,BodyType_Text_Messi,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Lean-,BodyType_Text_Short and Balanced,BodyType_Text_Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LaLiga Santander,League_Cat_Ligue 1,League_Cat_Major League Soccer,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Other,Nation_Cat_Spain
0,Pelé,98,CAM,Explosive,3270000.0,5,4,H,M,95.0,96,93,96,60,76,3928,516,2513,173,Unique,70,1,Icons,Other,CF,ST,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0
1,Lionel Messi,98,RW,Controlled,4350000.0,4,4,H,L,93.0,98,97,99,40,77,5575,504,2469,169,Messi,67,0,Ligue 1,Argentina,RM,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Lionel Messi,98,ST,Controlled,4640000.0,4,4,H,L,94.0,97,96,99,40,79,701,505,2455,169,Messi,67,0,Ligue 1,Argentina,RM,RW,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Karim Benzema,97,CF,Controlled,1850000.0,4,5,H,M,92.0,97,90,94,45,90,4536,508,2449,185,Average,81,0,LaLiga Santander,France,ST,NaN,NaN,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,Kylian Mbappé,97,ST,Controlled,9750000.0,5,4,H,L,99.0,96,88,98,44,87,3548,512,2455,182,Unique,73,0,Ligue 1,France,CF,LW,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0


In [86]:
# dropping the repeated variables
futbin_data = futbin_data.drop(['Main_Position', 'Run_Style', 'Attack_Workrate', 
                        'Defense_Workrate', 'BodyType_Text', 'League_Cat', 'Nation_Cat'], 1)

In [87]:
futbin_data.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Weight,Club_Hero,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Explosive,Run_Style_Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_High & Average,BodyType_Text_High & Average+,BodyType_Text_High & Lean,BodyType_Text_High & Stocky,BodyType_Text_Lean,BodyType_Text_Messi,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Lean-,BodyType_Text_Short and Balanced,BodyType_Text_Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LaLiga Santander,League_Cat_Ligue 1,League_Cat_Major League Soccer,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Other,Nation_Cat_Spain
0,Pelé,98,3270000.0,5,4,95.0,96,93,96,60,76,3928,516,2513,173,70,1,CF,ST,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0
1,Lionel Messi,98,4350000.0,4,4,93.0,98,97,99,40,77,5575,504,2469,169,67,0,RM,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Lionel Messi,98,4640000.0,4,4,94.0,97,96,99,40,79,701,505,2455,169,67,0,RM,RW,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Karim Benzema,97,1850000.0,4,5,92.0,97,90,94,45,90,4536,508,2449,185,81,0,ST,NaN,NaN,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,Kylian Mbappé,97,9750000.0,5,4,99.0,96,88,98,44,87,3548,512,2455,182,73,0,CF,LW,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0


#### Alt positins filling with missing value, and use it as a category

##### Alt_Pos_1

In [88]:
futbin_data.Alt_Pos_1.isnull().sum()

2028

In [89]:
futbin_data.Alt_Pos_1.fillna('missing', inplace=True)

In [90]:
futbin_data.Alt_Pos_1.isnull().sum()

0

In [91]:
futbin_data.Alt_Pos_1.value_counts()

missing    2028
CF          780
CM          525
RM          444
RWB         435
LM          366
CDM         364
LWB         360
RB          286
CAM         208
LB          176
RW          147
CB          127
ST           81
LW           70
Name: Alt_Pos_1, dtype: int64

##### Alt_Pos_2

In [92]:
futbin_data.Alt_Pos_2.isnull().sum()

4563

In [93]:
futbin_data.Alt_Pos_2.fillna('missing', inplace=True)

In [94]:
futbin_data.Alt_Pos_2.isnull().sum()

0

In [95]:
futbin_data.Alt_Pos_2.value_counts()

missing    4563
LW          284
RW          270
LM          206
CAM         196
ST          182
CF          162
RM          132
CM          130
LB           89
CB           73
LWB          68
CDM          33
RB            9
Name: Alt_Pos_2, dtype: int64

##### Alt_Pos_3

In [96]:
futbin_data.Alt_Pos_3.isnull().sum()

6009

In [97]:
futbin_data.Alt_Pos_3.fillna('missing', inplace=True)

In [98]:
futbin_data.Alt_Pos_3.isnull().sum()

0

In [99]:
futbin_data.Alt_Pos_3.value_counts()

missing    6009
LW          146
RW           72
ST           51
CF           45
LM           25
CDM          14
RM           11
CAM           9
LWB           9
LB            3
CM            3
Name: Alt_Pos_3, dtype: int64

In [100]:
futbin_data.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Weight,Club_Hero,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Explosive,Run_Style_Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_High & Average,BodyType_Text_High & Average+,BodyType_Text_High & Lean,BodyType_Text_High & Stocky,BodyType_Text_Lean,BodyType_Text_Messi,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Lean-,BodyType_Text_Short and Balanced,BodyType_Text_Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LaLiga Santander,League_Cat_Ligue 1,League_Cat_Major League Soccer,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Other,Nation_Cat_Spain
0,Pelé,98,3270000.0,5,4,95.0,96,93,96,60,76,3928,516,2513,173,70,1,CF,ST,missing,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0
1,Lionel Messi,98,4350000.0,4,4,93.0,98,97,99,40,77,5575,504,2469,169,67,0,RM,missing,missing,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Lionel Messi,98,4640000.0,4,4,94.0,97,96,99,40,79,701,505,2455,169,67,0,RM,RW,missing,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Karim Benzema,97,1850000.0,4,5,92.0,97,90,94,45,90,4536,508,2449,185,81,0,ST,missing,missing,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,Kylian Mbappé,97,9750000.0,5,4,99.0,96,88,98,44,87,3548,512,2455,182,73,0,CF,LW,missing,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0


#### Create categorical columns for the alt pos columns

In [101]:
# Creating a dummy variable for the alt_pos categorical variables and dropping the first one.
dummy_altpos = pd.get_dummies(futbin_data[['Alt_Pos_1', 'Alt_Pos_2', 'Alt_Pos_3']], drop_first=True)

# Adding the results to the master dataframe
futbin_data = pd.concat([futbin_data, dummy_altpos], axis=1)

In [102]:
futbin_data.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Weight,Club_Hero,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Explosive,Run_Style_Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_High & Average,BodyType_Text_High & Average+,BodyType_Text_High & Lean,BodyType_Text_High & Stocky,BodyType_Text_Lean,BodyType_Text_Messi,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Lean-,BodyType_Text_Short and Balanced,BodyType_Text_Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LaLiga Santander,League_Cat_Ligue 1,League_Cat_Major League Soccer,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Other,Nation_Cat_Spain,Alt_Pos_1_CB,Alt_Pos_1_CDM,Alt_Pos_1_CF,Alt_Pos_1_CM,Alt_Pos_1_LB,Alt_Pos_1_LM,Alt_Pos_1_LW,Alt_Pos_1_LWB,Alt_Pos_1_RB,Alt_Pos_1_RM,Alt_Pos_1_RW,Alt_Pos_1_RWB,Alt_Pos_1_ST,Alt_Pos_1_missing,Alt_Pos_2_CB,Alt_Pos_2_CDM,Alt_Pos_2_CF,Alt_Pos_2_CM,Alt_Pos_2_LB,Alt_Pos_2_LM,Alt_Pos_2_LW,Alt_Pos_2_LWB,Alt_Pos_2_RB,Alt_Pos_2_RM,Alt_Pos_2_RW,Alt_Pos_2_ST,Alt_Pos_2_missing,Alt_Pos_3_CDM,Alt_Pos_3_CF,Alt_Pos_3_CM,Alt_Pos_3_LB,Alt_Pos_3_LM,Alt_Pos_3_LW,Alt_Pos_3_LWB,Alt_Pos_3_RM,Alt_Pos_3_RW,Alt_Pos_3_ST,Alt_Pos_3_missing
0,Pelé,98,3270000.0,5,4,95.0,96,93,96,60,76,3928,516,2513,173,70,1,CF,ST,missing,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,Lionel Messi,98,4350000.0,4,4,93.0,98,97,99,40,77,5575,504,2469,169,67,0,RM,missing,missing,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,Lionel Messi,98,4640000.0,4,4,94.0,97,96,99,40,79,701,505,2455,169,67,0,RM,RW,missing,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Karim Benzema,97,1850000.0,4,5,92.0,97,90,94,45,90,4536,508,2449,185,81,0,ST,missing,missing,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,Kylian Mbappé,97,9750000.0,5,4,99.0,96,88,98,44,87,3548,512,2455,182,73,0,CF,LW,missing,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [103]:
# dropping the repeated variables
futbin_data = futbin_data.drop(['Alt_Pos_1', 'Alt_Pos_2', 'Alt_Pos_3'], 1)

In [104]:
futbin_data.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Weight,Club_Hero,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Explosive,Run_Style_Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_High & Average,BodyType_Text_High & Average+,BodyType_Text_High & Lean,BodyType_Text_High & Stocky,BodyType_Text_Lean,BodyType_Text_Messi,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Lean-,BodyType_Text_Short and Balanced,BodyType_Text_Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LaLiga Santander,League_Cat_Ligue 1,League_Cat_Major League Soccer,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Other,Nation_Cat_Spain,Alt_Pos_1_CB,Alt_Pos_1_CDM,Alt_Pos_1_CF,Alt_Pos_1_CM,Alt_Pos_1_LB,Alt_Pos_1_LM,Alt_Pos_1_LW,Alt_Pos_1_LWB,Alt_Pos_1_RB,Alt_Pos_1_RM,Alt_Pos_1_RW,Alt_Pos_1_RWB,Alt_Pos_1_ST,Alt_Pos_1_missing,Alt_Pos_2_CB,Alt_Pos_2_CDM,Alt_Pos_2_CF,Alt_Pos_2_CM,Alt_Pos_2_LB,Alt_Pos_2_LM,Alt_Pos_2_LW,Alt_Pos_2_LWB,Alt_Pos_2_RB,Alt_Pos_2_RM,Alt_Pos_2_RW,Alt_Pos_2_ST,Alt_Pos_2_missing,Alt_Pos_3_CDM,Alt_Pos_3_CF,Alt_Pos_3_CM,Alt_Pos_3_LB,Alt_Pos_3_LM,Alt_Pos_3_LW,Alt_Pos_3_LWB,Alt_Pos_3_RM,Alt_Pos_3_RW,Alt_Pos_3_ST,Alt_Pos_3_missing
0,Pelé,98,3270000.0,5,4,95.0,96,93,96,60,76,3928,516,2513,173,70,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,Lionel Messi,98,4350000.0,4,4,93.0,98,97,99,40,77,5575,504,2469,169,67,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,Lionel Messi,98,4640000.0,4,4,94.0,97,96,99,40,79,701,505,2455,169,67,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Karim Benzema,97,1850000.0,4,5,92.0,97,90,94,45,90,4536,508,2449,185,81,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,Kylian Mbappé,97,9750000.0,5,4,99.0,96,88,98,44,87,3548,512,2455,182,73,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### 4. Export final dataset

In [105]:
csv_futbin = futbin_data.to_csv('futbin.csv', index = False)
print('\nCSV String:\n', csv_futbin)


CSV String:
 None
